# scWGS Mutational Signature Analysis

In [ ]:
cat("\nR version :", paste(R.Version()$major, R.Version()$minor, sep="."), "\n")
.libPaths()
.libPaths(c("/autofs/projects-t1/devel/ashetty/R/x86_64-pc-linux-gnu-library/3.6","/usr/local/packages/r-3.6.0/lib64/R/library"))
.libPaths()

In [ ]:
# Load Packages
suppressMessages( library("BSgenome") )
suppressMessages( library("BSgenome.Mmusculus.UCSC.mm10") )
suppressMessages( library("MutationalPatterns") )
suppressMessages( library("NMF") )
suppressMessages( library("ggplot2") )
suppressMessages( library("gridExtra") )
suppressMessages( library("RColorBrewer") )

In [ ]:
# Sample Info
sINFO = "BBRAG_P02.scWGS.sample.info"
oINFO = read.delim(sINFO, sep="\t", header=T, stringsAsFactor=F)
colnames(oINFO) = c("Sample.ID", "Condition", "VCF.File")
rownames(oINFO) = oINFO$Sample.ID
print(dim(oINFO))
print(head(oINFO))

# VCF Files
aFiles = oINFO$VCF.File
cat("\nNumber of VCF Files :", length(aFiles), "\n")

In [ ]:
# BSgenome Chromosome Names
sREF = BSgenome.Mmusculus.UCSC.mm10
aSeqNames = seqnames(sREF)
print(aSeqNames)
cat("\n"); print(standardChromosomes(sREF))

In [ ]:
sRDS = "BBRAG_P02.sWGS.sample.v20200910.mutation_type.RData"
if( file.exists(sRDS) ) {
    # Loading R Image
    lnames = load(file = sRDS)
    print(lnames)
    cat("\n")
} else {
    # Read VCF as GRanges
    aSID = oINFO$Sample.ID
    sREF = "BSgenome.Mmusculus.UCSC.mm10"
    oVCF.gr = read_vcfs_as_granges(aFiles, aSID, sREF)
    
    # Summary of VCFs as GRanges
    print(summary(oVCF.gr))
    cat("\n"); print(seqnames(oVCF.gr))
    
    # Mutation Type Occurences
    oMutTypes = mut_type_occurrences(vcf_list = oVCF.gr, ref_genome = sREF)
    print(dim(oMutTypes))
    print(head(oMutTypes))
    
    cat("\n### Save R Image ###\n")
    sRDS = "BBRAG_P02.sWGS.sample.v20200910.analysis.RData"
    save.image(file = sRDS)
}

In [ ]:
print(dim(oMutTypes))
cat("\n"); print(head(oMutTypes))

In [ ]:
# Sample Info
oINFO = read.delim(sINFO, sep="\t", header=T, stringsAsFactor=F)
colnames(oINFO) = c("Sample.ID", "Condition", "VCF.File")
rownames(oINFO) = oINFO$Sample.ID
print(dim(oINFO))
cat("\n"); print(head(oINFO))

# Conditions
aConditions = oINFO$Condition
aConditions = factor(aConditions, levels = unique(oINFO$Condition))
cat("\n"); print(table(aConditions))

In [ ]:
plot_spectrum(oMutTypes, by = oINFO$Sample.ID, CT = TRUE, legend = TRUE, colors = brewer.pal(n=7, "Dark2"))

In [ ]:
plot_spectrum(oMutTypes, by = aConditions, CT = TRUE, legend = TRUE, colors = brewer.pal(n=7, "Dark2"))

In [ ]:
sOutFile = "BBRAG_P02.sWGS.sample.v20200910.trinucleotide_profile.txt"
if( file.exists(sRDS) ) {
    # Loading Mutational Profile
    oMutMatrix = read.delim(sOutFile, sep="\t", header=T, stringsAsFactor=F)
    rownames(oMutMatrix) = oMutMatrix$Trinucleotide.Change
    oMutMatrix = oMutMatrix[,-1]
    print(dim(oMutMatrix))
    print(head(oMutMatrix))
    cat("\n")
} else {
    # 96 Mutational Profile
    oMutMatrix = mut_matrix(vcf_list = oVCF.gr, ref_genome = sREF)
    print(dim(oMutMatrix))
    print(head(oMutMatrix))
    
    sOutFile = "BBRAG_P02.sWGS.sample.v20200910.trinucleotide_profile.txt"
    cnames = colnames(oMutMatrix)
    oOUT = cbind(rownames(oMutMatrix), oMutMatrix)
    colnames(oOUT) = c("Trinucleotide.Change", cnames)
    print(dim(oOUT))
    write.table(oOUT, file=sOutFile, append=F, quote=F, sep="\t", eol="\n", na="NA", row.names=F, col.names=T)
}

In [ ]:
plot_96_profile(oMutMatrix, colors = brewer.pal(n=6, "Dark2"), ymax = 0.05, condensed = TRUE)

In [ ]:
oMutMatrix = oMutMatrix + 0.0001
print(dim(oMutMatrix))
print(head(oMutMatrix))

In [ ]:
# COSMIC Mutational Signatures (N = 30)
sURL = paste("https://cancer.sanger.ac.uk/cancergenome/assets/", "signatures_probabilities.txt", sep = "")
oCOSMIC.n30 = read.table(sURL, sep = "\t", header = TRUE)
print(dim(oCOSMIC.n30))

aOrder = match(row.names(oMutMatrix), oCOSMIC.n30$Somatic.Mutation.Type)
oRefSig = oCOSMIC.n30[as.vector(aOrder),]
row.names(oRefSig) = oRefSig$Somatic.Mutation.Type
oRefSig = as.matrix(oRefSig[,4:33])
print(dim(oRefSig))
cat("\n"); print(oRefSig[1:6,1:6])

In [ ]:
oRefSig.hclust = cluster_signatures(oRefSig, method = "average")
oRefSig.order = colnames(oRefSig)[oRefSig.hclust$order]

# Sample-level Similarity to COSMIC Mutational Signatures
oCosSim = cos_sim_matrix(oMutMatrix, oRefSig)
print(dim(oCosSim))
cat("\n"); print(oCosSim[1:6,1:6])

In [ ]:
plot_cosine_heatmap(oCosSim, col_order = oRefSig.order, cluster_rows = TRUE)

In [ ]:
# COSMIC Mutational Signatures contributing to each Sample
oFIT = fit_to_signatures(oMutMatrix, oRefSig)
print(dim(oFIT$contribution))
cat("\n"); print(head(oFIT$contribution))

nSelect = 12
bThresh = sort(apply(oFIT$contribution, 1, max), decreasing = TRUE)[(nSelect + 1)]

cat("\nMax Contribution Threshold:", bThresh, "\n")
aSelect = which(apply(oFIT$contribution, 1, max) > bThresh)
cat("\nNumber of Selected COSMIC Signatures :", length(aSelect), "\n")
cat("\nSelected COSMIC Signatures :", aSelect, "\n")

In [ ]:
oP1 = plot_contribution(oFIT$contribution[aSelect,], oRefSig[,aSelect], mode = "relative", coord_flip = TRUE)
oP2 = plot_contribution(oFIT$contribution[aSelect,], oRefSig[,aSelect], mode = "absolute", coord_flip = TRUE)
grid.arrange(oP1, oP2)

In [ ]:
plot_contribution_heatmap(oFIT$contribution[aSelect,], cluster_samples = TRUE,, method = "complete")